# Intégration des données dans un data warehouse
# BUT SD 2
##### Par: Tim Touviot, Éloïse Serrand, Julien Renoult, Dalyop Yop
###### 24/11/2023  - 04/122023

# Initialisation

Ici, nous allons importer les packages nécessaires au bon fonctionnement du programme. On associera aussi les chemins nécessaires à la lecture des données en s'aidant de la fonction _getcwd()_ de la librairie __os__ permettant de connaître l'emplacement actuel où on exécute le programme.

In [1]:
# Chargement des librairies
import csv
import pandas as pd
import os
import glob
import re as r

# Adapter les chemins si nécessaires :
chemin_pollution = "releves_sheffields_22_10_2022/Pollution/"
chemin_meteo = "releves_sheffields_22_10_2022/Meteo/"
os.getcwd()

'/home/serrand222/Documents/BUT2/SAE_INT'

## Lecture des fichiers sur la pollution de l'air (POLLUTION)

Pour pouvoir faire la lecture des fichiers de données sur la pollution, nous allons créer deux fonctions permettant de réunir toutes ces données. La première fonction _extraction_csv_ permet de retourner une trame pandas par la lecture d'un fichier csv avec la librairie __pandas__ en précisant bien sûr différentes options permettant une lecture plus appropriées. La deuxième fonction _chargement_des_donnees_ va permettre d'automatiser cela et de lire tous les fichiers csv sur un même dossier en renvoyant une liste de trames. De plus, les fichiers pollutions ont tous la même structure avec la présence des noms de variables sur la première ligne, du séparateur décimal qui est une virgule et de deux colonnes.

In [2]:
def extraction_csv(fichier_source):
    """
    Permet d'extraire des données d'un fichier csv à l'aide de :
       - fichier_source : représente le nom du fichier et le chemin où le trouver
    """
    
    path = fichier_source
    
    trame = pd.read_csv(path,
                       sep =";",
                       header = 0,
                       decimal = ",")
    return trame


def chargement_des_donnees(repertoire_destination):
    """
    Cette fonction permet de créer une liste de dataframe pandas à l'aide de :
    - repertoire_destination : représente le chemin ou sont conservés les fichiers csv à lire
    
    Cela permet donc de lire plusieurs fichiers excel et de les conserver dans une liste.
    """
    liste = []
    repertoire_initial = os.getcwd()
    les_fichiers = glob.glob(repertoire_destination+'*.csv')
    for fichier_courant in les_fichiers:
        donnees = extraction_csv(fichier_courant)
        liste.append(donnees)
    
    return liste

## Application des fonctions (POLLUTION)

Après la création de ces fonctions, nous allons récupérer les données de tous les fichiers à l'aide des deux fonctions qui permettent d'avoir une liste de trame. À la suite de ce chargement des données, nous allons préparer ces données à l'aide de cet algorithme ci-dessous pour avoir cette structure :

| Datetime | Station | Type | Unite | Valeur |
| ---- | ------- | ---- | ----- | ------ |

In [3]:
# Lecture des fichiers de données pollution
donnees_pollution = chargement_des_donnees(chemin_pollution)

print(donnees_pollution[0])

                   DateTime  Firvale, NO2, DIF[M], Value (ppb)
0       2000-01-01 01:00:00                                NaN
1       2000-01-01 02:00:00                                NaN
2       2000-01-01 03:00:00                                NaN
3       2000-01-01 04:00:00                                NaN
4       2000-01-01 05:00:00                                NaN
...                     ...                                ...
200107  2022-10-29 20:00:00                          16.186188
200108  2022-10-29 21:00:00                          22.301208
200109  2022-10-29 22:00:00                          19.585815
200110  2022-10-29 23:00:00                          12.697296
200111  2022-10-30 00:00:00                           9.851547

[200112 rows x 2 columns]


In [4]:
# Séparateur : ", " (cas : firewal,...) ou " " (cas : groundhog2) dans la deuxième nom de colonne

for ind in range(len(donnees_pollution)):
    
    # Récupération des noms de colonnes
    #print(donnees_pollution[ind].columns)
    
    if ", " in donnees_pollution[ind].columns[1]:
        # Dans le cas ici, on a un séparateur ", " avec quatres éléments dont trois qu'on doit récupérer
        liste = donnees_pollution[ind].columns[1].split(", ")
        n=3
        
    else:
        # Dans le cas ici, on a un séparateur " " avec trois éléments dont on doit récupérer
        liste = donnees_pollution[ind].columns[1].split(" ")
        n=2
    
    # Nom de la station
    station = liste[0]

    # Récupération de l'UNITÉ de mesure
    indices_car = r.search("\(\D*\d*\)", liste[n]).span()

    # Association du TYPE de mesure
    type_m = liste[1]
    
    # Association de l'UNITÉ de mesure
    mesure_m = liste[n][indices_car[0]+1:indices_car[1]-1]
    
    # Récupération du nombre de ligne du jeu de données actuel
    rows = len(donnees_pollution[ind].axes[0])
    
    # Ajout de la colonne STATION avec le nom de la station
    donnees_pollution[ind].insert(loc=1, column = "Station", 
                                  value = [station.lower()]*rows)

    # Ajout de la colonne TYPE de mesure avec le type_m
    donnees_pollution[ind].insert(loc=2, column = "Type", 
                                  value = [type_m]*rows)
    
    # Ajout de la colonne UNITÉ de mesure avec la mesure_m
    donnees_pollution[ind].insert(loc=3, column = "Unite",
                                 value = [mesure_m]*rows)
    
    # Modification du nom de colonne Values en Valeur
    donnees_pollution[ind].rename(columns = {
    
    donnees_pollution[ind].columns[4] : "Valeur"
    }, inplace = True)
    
    # Supprimer les valeurs manquantes
    donnees_pollution[ind] = donnees_pollution[ind].dropna()
    
    # Pouvoir regarder chaque jeu de données
    #print(donnees_pollution[ind])

# Vérification du succès de l'opération
donnees_pollution[0]

DateTime  Station Type Unite     Valeur
151032  2017-03-25 01:00:00  firvale  NO2   ppb  20.160400
151033  2017-03-25 02:00:00  firvale  NO2   ppb  19.264313
151034  2017-03-25 03:00:00  firvale  NO2   ppb  19.824310
151035  2017-03-25 04:00:00  firvale  NO2   ppb  21.840393
151036  2017-03-25 05:00:00  firvale  NO2   ppb  20.832306
...                     ...      ...  ...   ...        ...
200107  2022-10-29 20:00:00  firvale  NO2   ppb  16.186188
200108  2022-10-29 21:00:00  firvale  NO2   ppb  22.301208
200109  2022-10-29 22:00:00  firvale  NO2   ppb  19.585815
200110  2022-10-29 23:00:00  firvale  NO2   ppb  12.697296
200111  2022-10-30 00:00:00  firvale  NO2   ppb   9.851547

[36520 rows x 5 columns]

## Fusion des données (POLLUTION)

Vu que les données ont maintenant la même structure, ils ne manquent plus qu'à faire la fusion de toutes ces trames en une seule trame pandas que nous allons nommer _donnees_pollution_. Pour cela, nous allons utiliser une fonction de la librairie __pandas__ qui est _contact_.

In [5]:
donnees_pollution = pd.concat(donnees_pollution)

## Aperçu des données (POLLUTION)

In [8]:
# Vérification du succès de la fusion des données
print(donnees_pollution.head(n = 10))

                   DateTime  Station Type Unite     Valeur
151032  2017-03-25 01:00:00  firvale  NO2   ppb  20.160400
151033  2017-03-25 02:00:00  firvale  NO2   ppb  19.264313
151034  2017-03-25 03:00:00  firvale  NO2   ppb  19.824310
151035  2017-03-25 04:00:00  firvale  NO2   ppb  21.840393
151036  2017-03-25 05:00:00  firvale  NO2   ppb  20.832306
151037  2017-03-25 06:00:00  firvale  NO2   ppb  26.208496
151038  2017-03-25 07:00:00  firvale  NO2   ppb  30.240601
151039  2017-03-25 08:00:00  firvale  NO2   ppb  26.208496
151040  2017-03-25 09:00:00  firvale  NO2   ppb  18.480408
151041  2017-03-25 10:00:00  firvale  NO2   ppb  16.464294


## Lecture des fichiers sur les données météorologiques (MÉTÉO) 

Pour pouvoir faire la lecture des fichiers de données météorologiques, nous allons créer deux nouvelles fonctions permettant de réunir ces données. La première fonction _extraction_csv2_ permet comme l'autre de retourner une trame par la lecture d'un fichier csv avec la librairie __pandas__ en précisant d'autres options nécessaires à la lecture. La deuxième fonction _lecture_fichier_ va permettre d'automatiser cela et de lire tous les fichiers csv sur un même dossier et d'en renvoyer une trame pandas réunissant les différentes colonnes contenant les valeurs numériques. Cependant, les fichiers météorologiques n'ont pas tous la même structure, car certains fichiers csv ont plus de deux colonnes ce qu'il oblige à faire plus attention à la lecture de ces données. On peut ajouter aussi que les données ne sont faites que par une seule station (Athletic Stadium), ce qui permet d'ajouter la colonne nom de station plus facilement.

In [9]:
def extraction_csv2(fichier_source):
    """
    Prends le chemin d'un fichier csv et transforme les données du fichier en trame avec comme arguement :
    
    - fichier_source : chemin relatif menant au fichier csv
    
    """
    trame = pd.read_csv(fichier_source, sep=";", header = 0, encoding = "UTF-8", decimal=',')
    return trame

def lecture_fichier(repertoire):
    """
    Travaille avec la fonction extraction_csv2 afin de concatener les données dans une seule trame. 
    Elle a pour seul argument :
    
    - repertoire : chemin relatif menant au dossier contenant les fichiers csv
    
    """
    fichiers = glob.glob(os.path.join(repertoire,"*.csv"))
    dataset = pd.concat(map(extraction_csv2, fichiers))
    return dataset 

## Application des fonctions (MÉTÉO)

Après la création de ces deux fonctions, nous allons maintenant procéder à la lecture des fichiers csv météorologiques en les réunissant dans une seule trame pandas. Après que l'importation des données soit faite, nous allons préparer les données telles que la trame finale doit nous donner cette structure :

|DateTime | Station | Type | Unite| Valeur |
| ------- | --- | ------  | ----- | ----- |

In [10]:
# Lecture des fichiers csv
df_meteo = lecture_fichier(chemin_meteo)

# Vérification du succès de l'opération
print(df_meteo)

                   DateTime  DiffTemp 2-8 m (deg.C)  DiffTemp 8-22 m (deg.C)  \
0       2000-01-01 01:00:00               -0.074000                      NaN   
1       2000-01-01 02:00:00               -0.166000                      NaN   
2       2000-01-01 03:00:00               -0.183000                      NaN   
3       2000-01-01 04:00:00               -1.014999                      NaN   
4       2000-01-01 05:00:00               -0.216000                      NaN   
...                     ...                     ...                      ...   
200035  2022-10-26 20:00:00                     NaN                      NaN   
200036  2022-10-26 21:00:00                     NaN                      NaN   
200037  2022-10-26 22:00:00                     NaN                      NaN   
200038  2022-10-26 23:00:00                     NaN                      NaN   
200039  2022-10-27 00:00:00                     NaN                      NaN   

        Athletic Stadium, Global Radiat

### Renommer des colonnes et ajout de la nouvelle colonne Station

Nous allons renommer les noms de colonnes pour pouvoir faire plus facilement la préparation des données. Ensuite, nous allons rajouter la colonne __Station__ en donnant comme valeur _athletic stadium_. 

In [12]:
# Noms de colonnes
print(df_meteo.columns)

# Renommer certaines colonnes qui ont cette syntaxe :
# 'Athletic Stadium, Type, 000[M], Value (Unite)'

# Création de la liste des noms de colonnes
liste_colonne = list(df_meteo.columns)

for ind in range(len(liste_colonne)) :
    if ", " in liste_colonne[ind]:
        
            # Séparation du nom de colonne par ", " pour récupérer plus facilement le type et l'unité
            liste = liste_colonne[ind].split(", ")
            
            # Récupération du type
            type_m = liste[1]
            
            # Récupération de l'unité
            unite_m = liste[3].split(" ")[1]
            
            # Renommer la colonne à l'aide des deux éléments
            df_meteo.rename(columns = {
                df_meteo.columns[ind] : type_m+" "+unite_m
            }, inplace = True)

# Création de la nouvelle colonne Station
df_meteo['Station'] = 'athletic stadium'

# Vérification du succès des deux opérations
print(df_meteo.columns)
print(df_meteo.head(n=2))


Index(['DateTime', 'DiffTemp 2-8 m (deg.C)', 'DiffTemp 8-22 m (deg.C)',
       'Global Radiation (W/m2)', 'Max Wind Speed 10 m (m/s)',
       'Max Wind Speed 24 m (m/s)', 'Pressure (hPa)', 'Rain (mm)',
       'Temp (deg.C)', 'Wind Direction 10 m (deg.M)',
       'Wind Direction 24 m (deg.M)', 'Wind Speed 10 m (m/s)',
       'Wind Speed 24 m (m/s)', 'Station'],
      dtype='object')
Index(['DateTime', 'DiffTemp 2-8 m (deg.C)', 'DiffTemp 8-22 m (deg.C)',
       'Global Radiation (W/m2)', 'Max Wind Speed 10 m (m/s)',
       'Max Wind Speed 24 m (m/s)', 'Pressure (hPa)', 'Rain (mm)',
       'Temp (deg.C)', 'Wind Direction 10 m (deg.M)',
       'Wind Direction 24 m (deg.M)', 'Wind Speed 10 m (m/s)',
       'Wind Speed 24 m (m/s)', 'Station'],
      dtype='object')
              DateTime  DiffTemp 2-8 m (deg.C)  DiffTemp 8-22 m (deg.C)  \
0  2000-01-01 01:00:00                  -0.074                      NaN   
1  2000-01-01 02:00:00                  -0.166                      NaN   

   G

### Transposition des noms de colonnes et conversion en numérique des valeurs

Après cette étape réussie, nous allons procéder à la transposition des noms de colonnes en ligne dans une nouvelle colonne nommée __Type (mesure)__ contenant à la fois le type et l'unité de mesure.

In [14]:
# Transposition des données
donnees_meteo = pd.melt(df_meteo, id_vars=["DateTime", "Station"], var_name="Type (mesure)", value_name="Value")

# Vérification du succès des deux opérations
print(donnees_meteo)
print(donnees_meteo['Type (mesure)'].unique())

                     DateTime           Station           Type (mesure)  \
0         2000-01-01 01:00:00  athletic stadium  DiffTemp 2-8 m (deg.C)   
1         2000-01-01 02:00:00  athletic stadium  DiffTemp 2-8 m (deg.C)   
2         2000-01-01 03:00:00  athletic stadium  DiffTemp 2-8 m (deg.C)   
3         2000-01-01 04:00:00  athletic stadium  DiffTemp 2-8 m (deg.C)   
4         2000-01-01 05:00:00  athletic stadium  DiffTemp 2-8 m (deg.C)   
...                       ...               ...                     ...   
19623163  2022-10-26 20:00:00  athletic stadium   Wind Speed 24 m (m/s)   
19623164  2022-10-26 21:00:00  athletic stadium   Wind Speed 24 m (m/s)   
19623165  2022-10-26 22:00:00  athletic stadium   Wind Speed 24 m (m/s)   
19623166  2022-10-26 23:00:00  athletic stadium   Wind Speed 24 m (m/s)   
19623167  2022-10-27 00:00:00  athletic stadium   Wind Speed 24 m (m/s)   

             Value  
0        -0.074000  
1        -0.166000  
2        -0.183000  
3        -1.014

### Supprimer les valeurs manquantes et séparation de colonne

La plupart des colonnes que nous avons instaurées sont là à part le type et l'unité de mesure. Ces valeurs se trouvent dans une même colonne nommée __Type (mesure)__. Pour ce faire, nous allons séparer les deux valeurs dans deux colonnes nommées __Type__ et __Unite__. Mais avant de faire cela, nous allons supprimer les lignes où il y a des valeurs manquantes pour traiter plus facilement les données.

In [15]:
# Supprimer les valeurs manquantes
donnees_meteo = donnees_meteo.dropna()

# Séparer les types et unités de mesure
donnees_meteo[["Type", "Unite"]] = donnees_meteo["Type (mesure)"].str.split('(', expand=True)

donnees_meteo["Unite"] = donnees_meteo["Unite"].str.replace("\)*", "", regex=True)

# Vérification du succès de l'opération
print(donnees_meteo.head(n=10))

              DateTime           Station           Type (mesure)     Value  \
0  2000-01-01 01:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.074000   
1  2000-01-01 02:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.166000   
2  2000-01-01 03:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.183000   
3  2000-01-01 04:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -1.014999   
4  2000-01-01 05:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.216000   
5  2000-01-01 06:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.229000   
6  2000-01-01 07:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.089000   
7  2000-01-01 08:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.089000   
8  2000-01-01 09:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.079000   
9  2000-01-01 10:00:00  athletic stadium  DiffTemp 2-8 m (deg.C) -0.053000   

              Type  Unite  
0  DiffTemp 2-8 m   deg.C  
1  DiffTemp 2-8 m   deg.C  
2  DiffTemp 2-8 m   deg.C  
3  DiffTemp 2-8 m   deg.C  
4

### Supprimer et renommer des colonnes

Après que toutes ces opérations soient réussies, on peut maintenant supprimer la colonne __Type (mesure)__ et renommer la colonne Value en nom de colonne Valeur.

In [16]:
# Supprimer la colonne Type (mesure)
donnees_meteo = donnees_meteo.drop('Type (mesure)', axis=1)

# Renommer la colonne Value
donnees_meteo.rename(columns = {
    
    donnees_meteo.columns[2] : "Valeur"
    }, inplace = True)

# Vérification du succès de l'opération
donnees_meteo.head(n = 10)

DateTime           Station    Valeur             Type  Unite
0  2000-01-01 01:00:00  athletic stadium -0.074000  DiffTemp 2-8 m   deg.C
1  2000-01-01 02:00:00  athletic stadium -0.166000  DiffTemp 2-8 m   deg.C
2  2000-01-01 03:00:00  athletic stadium -0.183000  DiffTemp 2-8 m   deg.C
3  2000-01-01 04:00:00  athletic stadium -1.014999  DiffTemp 2-8 m   deg.C
4  2000-01-01 05:00:00  athletic stadium -0.216000  DiffTemp 2-8 m   deg.C
5  2000-01-01 06:00:00  athletic stadium -0.229000  DiffTemp 2-8 m   deg.C
6  2000-01-01 07:00:00  athletic stadium -0.089000  DiffTemp 2-8 m   deg.C
7  2000-01-01 08:00:00  athletic stadium -0.089000  DiffTemp 2-8 m   deg.C
8  2000-01-01 09:00:00  athletic stadium -0.079000  DiffTemp 2-8 m   deg.C
9  2000-01-01 10:00:00  athletic stadium -0.053000  DiffTemp 2-8 m   deg.C

### Changement d'ordre des colonnes

Pour terminer la préparation des données, nous allons changer l'ordre des colonnes pour avoir la même structure. Après ça, nous pouvons maintenant procéder à l'exportation des deux jeux de données.

In [17]:
# Changement d'ordre des colonnes
donnees_meteo = donnees_meteo[["DateTime", "Station", "Type", "Unite", "Valeur"]]

# Vérification du succès de l'opération
donnees_meteo.head(n = 10)

DateTime           Station             Type  Unite    Valeur
0  2000-01-01 01:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.074000
1  2000-01-01 02:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.166000
2  2000-01-01 03:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.183000
3  2000-01-01 04:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -1.014999
4  2000-01-01 05:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.216000
5  2000-01-01 06:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.229000
6  2000-01-01 07:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.089000
7  2000-01-01 08:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.089000
8  2000-01-01 09:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.079000
9  2000-01-01 10:00:00  athletic stadium  DiffTemp 2-8 m   deg.C -0.053000

# Exportation des deux jeux de données

In [19]:
# Exportation des données météorologiques
donnees_meteo.to_csv("meteo.csv")

# Exportation des données sur la pollution
donnees_pollution.to_csv("pollution.csv")